Import work

In [2]:
from tvDatafeed import TvDatafeed,Interval
import numpy as np
import pandas as pd

Functions for later usage

In [5]:
def getVolatility(data):
	return(round(np.std(data) * np.sqrt(252) * 100, 3))

def getDailyReturns(data):
	return(data.pct_change())

def getFIP(data):
	retPos = np.sum(data.pct_change()[1:] > 0)
	retNeg = np.sum(data.pct_change()[1:] < 0)
	return(retPos - retNeg)

def getSortino(data):
	return(round(np.sqrt(252) * data.mean()/data[data<0].std(), 3))

def getMaxDrawdown(data):
	cummRet = (data+1).cumprod()
	peak = cummRet.expanding(min_periods = 1).max()
	drawdown = (cummRet/peak) - 1
	return drawdown.min()

def getCalmar(data):
	return(data.mean()*252/abs(getMaxDrawdown(data)))

def getSharpe(data):
	return(round(np.sqrt(252) * data.mean()/data.std(), 3))

def getBeta(dfNifty, data12M):
	
	dailyReturns = getDailyReturns(pd.concat([dfNifty, data12M], axis = 1))[1:]
	dailyReturns.rename(columns={'close':'Nifty'}, inplace=True)
	var = dailyReturns.loc[:, 'Nifty'].var()
	cov = dailyReturns.cov()
	cols = cov.columns[1:]
	beta = []
	for k in cols:
		beta.append(round(cov.loc[k, 'Nifty']/var, 3))

	return beta

Download Data.

In [7]:
tv = TvDatafeed()
nifty15m =tv.get_hist('NIFTY','NSE',interval=Interval.in_15_minute,n_bars=2880)
nifty15m

,symbol,open,high,low,close,volume
datetime,,,,,,
2022-10-20 14:15:00,NSE:NIFTY,17506.75,17516.15,17495.15,17503.35,8755320.0
2022-10-20 14:30:00,NSE:NIFTY,17502.10,17549.15,17501.65,17530.45,10123580.0
2022-10-20 14:45:00,NSE:NIFTY,17530.35,17539.15,17520.60,17531.10,8428707.0
2022-10-20 15:00:00,NSE:NIFTY,17528.90,17567.25,17524.35,17567.20,25649370.0
2022-10-20 15:15:00,NSE:NIFTY,17567.35,17584.15,17562.65,17565.25,30249714.0
...,...,...,...,...,...,...
2023-04-11 13:00:00,NSE:NIFTY,17682.00,17682.75,17663.85,17667.60,10808755.0
2023-04-11 13:15:00,NSE:NIFTY,17668.35,17684.90,17665.45,17683.15,7849619.0
2023-04-11 13:30:00,NSE:NIFTY,17682.75,17695.65,17682.30,17694.30,10164744.0


In [10]:
# Create Yahoo symbol
df = pd.read_csv('./indices/foTest.csv')
df['Yahoo_Symbol'] = df.Symbol
df = df.set_index('Yahoo_Symbol')
symbol = list(df.index)
symbol

['ADANIENT',
 'BHARTIARTL',
 'CIPLA',
 'HDFCBANK',
 'ITC',
 'INFY',
 'JSWSTEEL',
 'RELIANCE',
 'SBIN',
 'TATAMOTORS']

In [13]:
close = []
for k in range(0, len(symbol)):
    print(k, end = ',', flush = True)
    _x = tv.get_hist(symbol[k],'NSE',interval=Interval.in_15_minute,n_bars=2880).close
    close = _x if len(close) == 0 else pd.concat([close, _x], axis = 1)
close

0,1,2,3,4,5,6,7,8,9,

,close,close,close,close,close,close,close,close,close,close
datetime,,,,,,,,,,
2022-10-20 14:15:00,3343.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-10-20 14:30:00,3346.95,791.10,1135.15,1449.00,348.00,1498.00,626.35,2503.50,554.75,398.90
2022-10-20 14:45:00,3352.90,792.80,1135.50,1448.00,348.40,1496.20,626.15,2502.20,554.10,398.15
2022-10-20 15:00:00,3354.05,792.55,1135.90,1448.75,349.05,1502.05,628.35,2501.90,556.00,398.00
2022-10-20 15:15:00,3365.00,791.45,1138.05,1449.00,350.75,1501.00,628.45,2498.90,555.50,398.50
...,...,...,...,...,...,...,...,...,...,...
2023-04-11 13:15:00,1799.65,773.15,905.05,1663.50,395.00,1398.50,712.45,2329.45,532.85,457.45
2023-04-11 13:30:00,1802.65,774.15,906.90,1660.50,396.10,1403.45,712.35,2331.90,533.95,457.25
2023-04-11 13:45:00,1809.60,773.95,907.05,1662.65,397.00,1404.15,711.80,2332.95,533.50,457.45


In [14]:
close.columns = symbol
close = close[1:]
close

,ADANIENT,BHARTIARTL,CIPLA,HDFCBANK,ITC,INFY,JSWSTEEL,RELIANCE,SBIN,TATAMOTORS
datetime,,,,,,,,,,
2022-10-20 14:30:00,3346.95,791.10,1135.15,1449.00,348.00,1498.00,626.35,2503.50,554.75,398.90
2022-10-20 14:45:00,3352.90,792.80,1135.50,1448.00,348.40,1496.20,626.15,2502.20,554.10,398.15
2022-10-20 15:00:00,3354.05,792.55,1135.90,1448.75,349.05,1502.05,628.35,2501.90,556.00,398.00
2022-10-20 15:15:00,3365.00,791.45,1138.05,1449.00,350.75,1501.00,628.45,2498.90,555.50,398.50
2022-10-21 09:15:00,3361.00,793.55,1143.95,1453.30,349.95,1499.35,622.15,2514.80,560.80,400.95
...,...,...,...,...,...,...,...,...,...,...
2023-04-11 13:15:00,1799.65,773.15,905.05,1663.50,395.00,1398.50,712.45,2329.45,532.85,457.45
2023-04-11 13:30:00,1802.65,774.15,906.90,1660.50,396.10,1403.45,712.35,2331.90,533.95,457.25
2023-04-11 13:45:00,1809.60,773.95,907.05,1662.65,397.00,1404.15,711.80,2332.95,533.50,457.45


Stats DataFrame updation work   


In [17]:
dfStats = pd.DataFrame()
dfStats['Volatility'] = getVolatility(getDailyReturns(close))
dfStats['Sharpe'] = getSharpe(getDailyReturns(close))
dfStats['FIP'] = getFIP(close)
dfStats['Sortino'] = getSortino(getDailyReturns(close))
dfStats['beta'] = getBeta(nifty15m.close, close)
dfStats['Close'] = round(close.iloc[-1], 2)
dfStats['ema50'] = round(close.ewm(span=50).mean().iloc[-1], 2)
dfStats['ema100'] = round(close.ewm(span=100).mean().iloc[-1], 2)
dfStats['ema200'] = round(close.ewm(span=200).mean().iloc[-1], 2)
dfStats

,Volatility,Sharpe,FIP,Sortino,beta,Close,ema50,ema100,ema200
ADANIENT,18.291,-0.204,-56,-0.238,3.169,1805.75,1791.50,1772.28,1759.07
BHARTIARTL,3.988,-0.032,-20,-0.041,0.746,772.75,770.88,767.51,763.56
CIPLA,4.311,-0.433,-106,-0.494,0.439,907.30,902.25,898.83,894.02
HDFCBANK,3.584,0.355,26,0.505,1.118,1663.35,1660.29,1650.95,1631.82
ITC,4.298,0.290,-14,0.366,0.717,397.00,391.53,388.85,385.94
INFY,4.131,-0.113,25,-0.149,1.107,1406.15,1415.65,1416.23,1412.37
JSWSTEEL,4.912,0.253,53,0.343,0.994,712.00,698.02,691.73,684.73
RELIANCE,3.513,-0.159,-48,-0.221,1.092,2332.65,2333.55,2327.98,2311.70
SBIN,4.546,-0.058,-56,-0.080,1.302,532.00,530.70,528.10,524.88
TATAMOTORS,5.783,0.236,-127,0.388,0.883,457.40,455.28,446.71,435.98


In [21]:
# Create Yahoo symbol
df = pd.read_csv('./indices/foTv.csv')
df['Yahoo_Symbol'] = df.Symbol
df = df.set_index('Yahoo_Symbol')
symbol = list(df.index)
symbol

['ABCAPITAL',
 'ABBOTINDIA',
 'ABFRL',
 'AMBUJACEM',
 'ADANIENT',
 'ASHOKLEY',
 'APOLLOHOSP',
 'ASIANPAINT',
 'AUBANK',
 'AUROPHARMA',
 'ALKEM',
 'ACC',
 'BAJAJ_AUTO',
 'ASTRAL',
 'BAJAJFINSV',
 'BAJFINANCE',
 'BALKRISIND',
 'BATAINDIA',
 'BEL',
 'BHARTIARTL',
 'BIOCON',
 'BOSCHLTD',
 'BPCL',
 'BRITANNIA',
 'BSOFT',
 'BHEL',
 'CANBK',
 'CANFINHOME',
 'CHAMBLFERT',
 'CIPLA',
 'COLPAL',
 'CONCOR',
 'COROMANDEL',
 'CROMPTON',
 'CUB',
 'CUMMINSIND',
 'DELTACORP',
 'EICHERMOT',
 'EXIDEIND',
 'FEDERALBNK',
 'GAIL',
 'GLENMARK',
 'GMRINFRA',
 'GNFC',
 'GODREJCP',
 'GRASIM',
 'GUJGASLTD',
 'HAL',
 'HDFC',
 'ABB',
 'HDFCBANK',
 'HDFCLIFE',
 'HEROMOTOCO',
 'HINDPETRO',
 'HONAUT',
 'ICICIBANK',
 'HINDALCO',
 'IDFC',
 'IDFCFIRSTB',
 'IGL',
 'INDHOTEL',
 'INDIGO',
 'INDUSINDBK',
 'INFY',
 'INTELLECT',
 'HINDUNILVR',
 'IOC',
 'IPCALAB',
 'IRCTC',
 'IBULHSGFIN',
 'ICICIGI',
 'ITC',
 'JINDALSTEL',
 'JKCEMENT',
 'L&TFH',
 'LALPATHLAB',
 'JSWSTEEL',
 'LT',
 'LTIM',
 'LTTS',
 'LUPIN',
 'M&M',
 'M&MFIN',


In [27]:
import time

CHUNK = 10
tv = TvDatafeed()
close = []
for k in range(0, len(symbol), CHUNK):
    print(k, end = ' ', flush = True)
    for i in range(k+1, k+10):
        if i <= len(symbol):
            print(i, end=" ", flush=True)
            _x = tv.get_hist(symbol[i],'NSE',interval=Interval.in_15_minute,n_bars=2880).close
            close = _x if len(close) == 0 else pd.concat([close, _x], axis = 1)
    time.sleep(1)
    # _symlist = symbol[k:k+CHUNK]
    # _x = yf.download(_symlist, start = dates['startDate'], progress = False)['Close']
    # close = _x if len(close) == 0 else pd.concat([close, _x], axis = 1)
    # time.sleep(.5)
close


0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 

KeyboardInterrupt: 

In [28]:
# _x = tv.get_hist(symbol[11],'NSE',interval=Interval.in_15_minute,n_bars=2880).close
# _x
close

,close,close,close,close,close,close,close,close,close,close,close,close,close,close,close,close,close,close
datetime,,,,,,,,,,,,,,,,,,
2022-10-25 14:15:00,18622.60,332.00,513.45,3304.00,146.50,4392.95,3094.50,575.45,520.55,2253.00,3655.55,1496.6625,1664.10,7131.10,1946.65,1805.25,104.55,801.40
2022-10-25 14:30:00,18602.00,332.75,514.00,3305.40,147.05,4398.95,3097.75,574.80,520.90,2252.65,3656.50,1497.5250,1665.65,7135.00,1948.10,1804.05,104.70,803.50
2022-10-25 14:45:00,18505.15,332.65,513.40,3308.10,147.90,4397.70,3096.35,575.35,520.20,2250.00,3657.85,1490.7750,1663.10,7124.95,1945.50,1797.25,105.35,802.40
2022-10-25 15:00:00,18496.45,338.45,512.30,3300.10,147.30,4403.20,3085.20,575.40,519.75,2247.65,3650.00,1490.2500,1658.90,7103.00,1948.75,1797.00,105.15,802.10
2022-10-25 15:15:00,18452.05,340.00,513.00,3295.05,147.00,4401.00,3082.50,574.25,519.30,2246.00,3655.10,1488.7500,1657.40,7097.00,1944.75,1795.00,105.00,800.50
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-04-13 14:15:00,22482.10,225.85,394.95,1871.00,138.15,4420.90,2805.90,676.35,563.65,1766.50,4289.95,1452.4000,1341.70,5912.00,2069.00,1397.00,101.15,768.55
2023-04-13 14:30:00,22499.90,225.85,393.55,1872.25,138.30,4402.35,2807.95,676.70,564.60,1765.95,4288.95,1447.8000,1343.10,5915.00,2068.90,1396.90,101.00,767.90
2023-04-13 14:45:00,22482.65,226.00,393.65,1870.80,138.50,4396.00,2810.70,677.95,565.50,1767.60,4288.90,1446.5000,1343.70,5915.00,2063.05,1396.60,100.95,768.20
